encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
# 获取当前文件所在的文件夹路径
if "__file__" in globals():
    # 获取__file__变量的值
    file_path = __file__
    # 获取当前文件所在的文件夹路径
    dir_now = os.path.dirname(file_path)
else:
    # 获取当前工作目录
    dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
# for method in [METHOD_MITHRAL, METHOD_PQ]:

In [3]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"

auto_train_change_nbits = False # 是否根据已运行的训练性能结果改变nbits自动训练，（train_sam_num取已训练的最大值）
auto_train_change_upcast = False # 是否根据已运行的训练性能结果改变upcast自动训练，（train_sam_num取已训练的最大值）

if auto_train_change_nbits:
    nbits_trained = 8
if auto_train_change_upcast:
    upcast_trained = 16
quantize_lut = False
nbits_goal = 8
upcast_goal = -1
lut_work_const = 1
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数
upcast_every = upcast_goal # 要运行的upcast

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train_change_nbits and not auto_train_change_upcast:
    ncodebooks = 512 # max:512
    ncentroids = 64
    train_sam_num = 1500 # 训练集样本数
elif auto_train_change_nbits:
    param2change = "nbits"
    param_trained = nbits_trained
    param_goal = nbits_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "upcast_every", 16)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])
elif auto_train_change_upcast:
    param2change = "upcast_every"
    param_trained = upcast_trained
    param_goal = upcast_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "nbits", 8)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])


In [5]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

In [6]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        upcast_every=upcast_every, bias_path=AMM_train_dirs["biaspath"],lut_work_const=-1)
elif method == METHOD_MITHRAL:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every, lut_work_const=lut_work_const)
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every)

running method:  Mithral
X.shape:  (1536000, 512)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    234.0869059716931
learn_multisplits(): returning loss:  0.053134926250301015
learn_multisplits(): initial loss:    56.06915243915815
learn_multisplits(): returning loss:  0.014999579081690235
learn_multisplits(): initial loss:    43.8877830100555
learn_multisplits(): returning loss:  0.0014792266697430739
learn_multisplits(): initial loss:    240.55207492965178
learn_multisplits(): returning loss:  0.03209313775313885
learn_multisplits(): initial loss:    20.55766972375736
learn_multisplits(): returning loss:  0.005070348722070902
learn_multisplits(): initial loss:    3004.784451439259
learn_multisplits(): returning loss:  0.7547575419726043
learn_multisplits(): initial loss:    176809.4756772358
learn_multisplits(): returning loss:  4959.193275446051
learn_multisplits(): initial loss:    2.195055887862099
learn_multisplits(): returning loss:  0.00016275077

/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:786: RuntimeWarning: divide by zero encountered in double_scalars
  scale = 254. / upper_val


split err: upper_val0.0 offset0.0
  oldvals:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  newvals:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
split err: upper_val0.0 offset0.0
  oldvals:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
  newvals:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
learn_multisplits(): returning loss:  0
learn_multisplits(): initial loss:    216.82348128804645
learn_multisplits(): returning loss:  0.030645471769242316
learn_multisplits(): initial loss:    1608.999346783471
learn_multisplits(): returning loss:  0.36781583144374574
learn_multisplits(): initial loss:    308.602566459616
learn_multisplits(): returning loss:  0.16131098774330344
learn_multisplits(): initial loss:    162842.8312222902
learn_multisplits(): returning loss:  572.8988561867791
learn_multisplits(): initial loss:    993.7594349821409
learn_multisplits(): returning loss:  0.4077318649161049
learn_multisplits(): initial loss: 

/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:1924: UserWarning: Persisting input arguments took 3.18s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  _learn_mithral_initialization(X, ncodebooks, ncentroids=ncentroids, pq_perm_algo='start', **kwargs)


X_res mse / X mse:  0.0023041503


/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../vquantizers.py:640: UserWarning: Persisting input arguments took 3.19s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  self.splits_lists, self.centroids = clusterize.learn_mithral(
/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../matmul.py:330: UserWarning: Persisting input arguments took 3.58s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the pr

In [7]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL or method == METHOD_PLUTO:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i_lwc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every, lut_work_const)), y_out_last_re)
elif method == METHOD_PQ or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)

[[ 0.52944803 -0.00773589  0.5436872  ...  0.25824946  0.01361134
   0.568839  ]
 [ 0.49708077  0.00137358  0.50875217 ...  0.3047716   0.04657958
   0.52348524]
 [ 0.5403061  -0.00466141  0.52756673 ...  0.25437042  0.01681941
   0.5338638 ]
 ...
 [ 0.48067966  0.10697985  0.2410889  ...  0.2732236   0.21450403
   0.30652335]
 [ 0.40263757 -0.10223573  0.18718801 ...  0.36928487  0.135007
   0.39773268]
 [ 0.36390364  0.14269428  0.2533929  ...  0.02572042  0.17600429
   0.29570398]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)


: 